<a href="https://colab.research.google.com/github/ymiu/my_first_Repository/blob/main/%E7%B5%B1%E8%A8%88API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# クラス実装 (※このセルは "estatapi.py" 等のファイル名でモジュール化可能)
import gzip
import urllib.request
import json
import pandas

class StatsData:
    """\
    e-Stat-APIにより統計データを取得し、これを保持する。

    Parameters
    ----------
    url : str
        APIのURLを指定する。
    appId : str
        APIのアプリケーションIDを指定する。
    data : dict, default None
        取得済みデータを指定する。
        未指定 (`None`) の場合、APIにより統計データを取得し、
        これを当該個体 (instance) の `data` プロパティに登録する。

    Examples
    --------
    個体生成から、取得したデータの代入迄を一気通貫で行う。

    >>> data = StatsData('url', 'appId').data

    上記を最大3段階の手続きに分割できる。

    >>> obj = StatsData('url', 'appId', data={})  # 1. 個体生成
    >>> obj.load()                          # 2. APIにより統計データを取得
    >>> data = obj.data                     # 3. 取得した統計データの代入
    """

    def __init__(self, url: str, appId: str, data: dict=None):
        self.url = url
        self.appId = appId
        self.data = data
        if data is None:
            self.load()

    def load(self):
        """\
        e-Stat-APIにより統計データを取得し、
        これを当該個体の `data` プロパティに登録する。
        """
        url = self.url
        url = url.replace('http:', 'https:')
        url = url.replace('appId=', 'appId=' + self.appId)
        if not '/app/json/' in url:
            url = url.replace('/app/', '/app/json/')

        request = urllib.request.Request(url, headers={'Accept-Encoding': 'gzip'}) 

        with urllib.request.urlopen(request) as r:
            gzipFile = gzip.GzipFile(fileobj=r) 
            self.data = json.loads(gzipFile.read().decode('utf8'))

class StatsDataForPandas(StatsData):
    """\
    e-Stat-APIにより統計データを取得し、これを保持する。
    当該個体 (instance) の `to_df` メソッドでデータフレームに変換できる。

    Parameters
    ----------
    url : str
        APIのURLを指定する。
    appId : str
        APIのアプリケーションIDを指定する。
    data : dict, default None
        取得済みデータを指定する。
        未指定 (`None`) の場合、APIにより統計データを取得し、
        これを当該個体の `data` プロパティに登録する。
    names : dict, default None
        置換辞書を指定する。
        未指定 (`None`) の場合、当該個体の `data` プロパティの値に基づいて
        置換辞書を生成し、これを当該個体の `names` プロパティに登録する。

    Examples
    --------
    個体生成から、置換済データフレームの取得迄を一気通貫で行う。

    >>> df = StatsDataForPandas('url', 'appId').to_df()

    上記を最大6段階の手続きに分割できる。

    >>> obj = StatsDataForPandas('url', 'appId', data={}, names={})  # 1. 個体生成
    >>> obj.load()                          # 2. APIにより統計データを取得
    >>> obj.make_names()                    # 3. 置換辞書を生成
    >>> df1 = obj.to_df(names={})           # 4. 未置換データフレームを取得
    >>> df2 = obj.code2name(df1)            # 5. code形式からname形式にデータ値置換
    >>> df3 = obj.id2name(df2)              # 6. id形式からname形式に列名置換

    id/code形式及びname形式を混在させる。

    >>> import numpy as np
    >>> import pandas as pd
    >>> obj = StatsDataForPandas('url', 'appId')
    >>> df_vcode = obj.to_df(names={})      # 未置換データフレームを取得
    >>> df_vname = obj.to_df()              # 置換済データフレームを取得
    >>> columns = np.empty((len(df_vcode.columns)*2), dtype=np.object)
    >>> columns[0::2] = df_vcode.columns
    >>> columns[1::2] = df_vname.columns
    >>> columns = pd.Series(columns).drop_duplicates()
    >>> df = pd.merge(df_vcode, df_vname).loc[:, columns]
    """

    def __init__(self, url: str, appId: str, data: dict=None, names: dict=None):
        super().__init__(url, appId, data)
        self.names = names
        if names is None:
            self.make_names()

    def make_names(self):
        """\
        当該個体の `data` プロパティに基づいて置換辞書 (`names`) を生成し、
        これを同個体の `names` プロパティに登録する。
        但し、同個体の `data` プロパティが空 (`{}`) の場合、処理を行わない。
        """
        if not self.data:
            return
        self.names = dict(id=dict(), code=dict())
        for obj in self.data['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ']:
            obj_id = '@' + obj['@id']
            if isinstance(obj['CLASS'], dict):
                self.names['id'][obj_id] = obj['@name']
                self.names['code'][obj_id] = {obj['CLASS']['@code']: obj['CLASS']['@name']}
            else: # list
                self.names['id'][obj_id] = obj['@name']
                self.names['code'][obj_id] = dict()
                for cat in obj['CLASS']:
                    self.names['code'][obj_id][cat['@code']] = cat['@name']

    def code2name(self, df: pandas.core.frame.DataFrame, names: dict=None) -> pandas.core.frame.DataFrame:
        """\
        指定された置換辞書 (`names`) の `code` キーに基づいて、
        各データ値をcode形式からname形式に置換・生成したデータフレームを返戻する。

        Parameters
        ----------
        df : pd.DataFrame
            置換の対象とするデータフレームを指定する。
        names : dict, default None
            置換辞書を指定する。
            置換辞書が未指定 (`None`) の場合、当該個体の `names` プロペティの値を使用する。
            置換辞書が空 (`{}`) の場合、データフレームの各値を置換しない。

        Returns
        -------
        pd.DataFrame
            生成データフレームを返戻する。
            但し、未置換の場合は指定した `df` を返戻する。
        """
        if names is None:
            names = self.names
        if 'code' in names:
            df = df.replace(names['code'])
        return df

    def id2name(self, df: pandas.core.frame.DataFrame, names: dict=None) -> pandas.core.frame.DataFrame:
        """\
        指定された置換辞書 (`names`) の `id` キーに基づいて、
        各列名をid形式からname形式に変換・生成したデータフレームを返戻する。

        Parameters
        ----------
        df : pd.DataFrame
            置換の対象とするデータフレームを指定する。
        names : dict, default None
            置換辞書を指定する。
            置換辞書が未指定 (`None`) の場合、当該個体の `names` プロペティの値を使用する。
            置換辞書が空 (`{}`) の場合、データフレームの列名を置換しない。

        Returns
        -------
        pd.DataFrame
            生成データフレームを返戻する。
            但し、未置換の場合は指定した `df` を返戻する。
        """
        if names is None:
            names = self.names
        if 'id' in names:
            df = df.copy()
            df.columns = pandas.Series(df.columns).replace(names['id'])
        return df

    def to_df(self, names: dict=None) -> pandas.core.frame.DataFrame:
        """\
        当該個体の `data` プロパティの値に基づき、データフレームを生成する。
        指定された置換辞書 (`names`) に基づき、生成データフレームの各データ値及び列名を変換する。

        Parameters
        ----------
        names : dict, default None
            生成データフレームの各値及び列名を変換する際に使用する置換辞書を指定する。
            置換辞書が未指定 (`None`) の場合、当該個体の `names` プロペティの値を使用する。
            置換辞書が空 (`{}`) の場合、データフレームの各値及び列名を置換しない。

        Returns
        -------
        pd.DataFrame
            生成データフレームを返戻する。
        """
        df = pandas.json_normalize(self.data['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE'])
        df = self.code2name(df, names)
        df = self.id2name(df, names)
        return df

import numpy as np
import pandas as pd

# e-Stat-APIのアプリケーションID
appId = '5ae4fcae8c5477c9e4474f0acb170003ff54ab4f'

# 16 平成22年国勢調査
url = 'http://api.e-stat.go.jp/rest/3.0/app/getStatsData?appId=&lang=J&statsDataId=0003410439&metaGetFlg=Y&cntGetFlg=N&explanationGetFlg=Y&annotationGetFlg=Y&sectionHeaderFlg=1'

# 統計データを取得
df = StatsDataForPandas(url, appId).to_df()

df

df = df.loc[(df["時間軸（調査年）"] == "2010年") & (df["@unit"] == "人") & (
    (df["年齢（各歳）年齢不詳，再掲有り2010"] == "総数") |
    (df["年齢（各歳）年齢不詳，再掲有り2010"] == "０歳") |
    (df["年齢（各歳）年齢不詳，再掲有り2010"] == "１歳") |
    (df["年齢（各歳）年齢不詳，再掲有り2010"] == "２歳")), :].astype({"$": np.int})

df

df = pd.pivot_table(data=df,
                    index="男女別2010",
                    columns="年齢（各歳）年齢不詳，再掲有り2010",
                    values="$").reindex(index=("総数（男女別）", "男", "女"),
                                        columns=("総数", "０歳", "１歳", "２歳"))
df.index = ("全国", "男", "女")
df.columns = ("総数（年齢）", "０歳", "１歳", "２歳")
with pd.option_context('display.float_format', "{:,.0f}".format):
    display(df.astype(np.float))

,総数（年齢）,０歳,１歳,２歳
全国,"128,057,352","1,045,975","1,045,417","1,074,194"
男,"62,327,737","535,357","534,800","549,618"
女,"65,729,615","510,618","510,617","524,576"
